# Deep Q Network with Pong

DQN was created by DeepMind researchers, achieving superhuman performance on many Atari games (https://deepmind.com/research/dqn/). What made the accomplishments of DQN even more impressive is that the DQN had nearly the same architecture and hyperparameter settings for each game (ie no game specific knowledge or set up) and the only inputs to the DQN were the screen pixels. The main parts of the DQN are Q learning, the neural net, experience replay, target network updating, and data/environment pre-processing.


In [138]:
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import gym
import numpy as np

from collections import deque
from gym import spaces
import cv2
cv2.ocl.setUseOpenCL(False)

In [139]:
# check and use GPU if available if not use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Experience Replay Buffer

Q learning is an off-policy method. In off-policy methods we often save samples and then train on them. DQN has a memory buffer that stores state, action, reward, done condition, and next state samples. In the training loop, DQN will sample from this memory buffer and train on the saved samples. The replay buffer is essentially a queue that we randomly sample from. When the queue gets full we replace the first element of the queue. 

In [140]:
# replay buffer from and some code based on https://github.com/sfujim/TD3

# create replay buffer of tuples of (state, next_state, action, reward, done)
class ReplayBuffer():
    def __init__(self, max_size=1e6):
        self.storage = []
        self.max_size = max_size
        self.ptr = 0

    def add(self, data):
        if len(self.storage) == self.max_size:
            self.storage[int(self.ptr)] = data
            self.ptr = (self.ptr + 1) % self.max_size
        else:
            self.storage.append(data)

    def sample(self, batch_size):
        ind = np.random.randint(0, len(self.storage), size=batch_size)
        x, y, u, r, d = [], [], [], [], []

        for i in ind: 
            X, Y, U, R, D = self.storage[i]
            x.append(np.array(X, copy=False))
            y.append(np.array(Y, copy=False))
            u.append(np.array(U, copy=False))
            r.append(np.array(R, copy=False))
            d.append(np.array(D, copy=False))

        return np.array(x), np.array(y), np.array(u).reshape(-1,1), np.array(r).reshape(-1,1), np.array(d).reshape(-1,1)

### Environment Wrappers

Wrappers modify gym environments. Wrappers can change the observation, reward, done, and info returns. Wrappers can also change the gym functions like step() and reset(). We include and use some common wrappers used in Atari games in the cells below. To use a wrapper, feed the gym env into the wrapper function like in the next cell. Sometimes problems require their own customized wrappers.

In [141]:
#env = gym.make('PongNoFrameskip-v4')
#env = wrap_deepmind(env, episode_life=True, clip_rewards=True, frame_stack=True, scale=False) #scale in DQN

In [142]:
# from: https://github.com/openai/baselines/baselines/common/atari_wrappers.py
# from: https://github.com/Officium/RL-Experiments/blob/master/src/common/wrappers.py 

class NoopResetEnv(gym.Wrapper):
    def __init__(self, env, noop_max=30):
        """Sample initial states by taking random number of no-ops on reset.
        No-op is assumed to be action 0.
        """
        gym.Wrapper.__init__(self, env)
        self.noop_max = noop_max
        self.override_num_noops = None
        self.noop_action = 0
        assert env.unwrapped.get_action_meanings()[0] == 'NOOP'

    def reset(self, **kwargs):
        """ Do no-op action for a number of steps in [1, noop_max]."""
        self.env.reset(**kwargs)
        if self.override_num_noops is not None:
            noops = self.override_num_noops
        else:
            noops = self.unwrapped.np_random.randint(1, self.noop_max + 1)
        assert noops > 0
        obs = None
        for _ in range(noops):
            obs, _, done, _ = self.env.step(self.noop_action)
            if done:
                obs = self.env.reset(**kwargs)
        return obs

    def step(self, ac):
        return self.env.step(ac)


class FireResetEnv(gym.Wrapper):
    def __init__(self, env):
        """Take action on reset for environments that are fixed until firing."""
        gym.Wrapper.__init__(self, env)
        assert env.unwrapped.get_action_meanings()[1] == 'FIRE'
        assert len(env.unwrapped.get_action_meanings()) >= 3

    def reset(self, **kwargs):
        self.env.reset(**kwargs)
        obs, _, done, _ = self.env.step(1)
        if done:
            self.env.reset(**kwargs)
        obs, _, done, _ = self.env.step(2)
        if done:
            self.env.reset(**kwargs)
        return obs

    def step(self, ac):
        return self.env.step(ac)


class EpisodicLifeEnv(gym.Wrapper):
    def __init__(self, env):
        """Make end-of-life == end-of-episode, but only reset on true game over.
        Done by DeepMind for the DQN and co. since it helps value estimation.
        """
        gym.Wrapper.__init__(self, env)
        self.lives = 0
        self.was_real_done = True

    def step(self, action):
        obs, reward, done, info = self.env.step(action)
        self.was_real_done = done
        # check current lives, make loss of life terminal,
        # then update lives to handle bonus lives
        lives = self.env.unwrapped.ale.lives()
        if 0 < lives < self.lives:
            # for Qbert sometimes we stay in lives == 0 condition for a few
            # frames so it's important to keep lives > 0, so that we only reset
            # once the environment advertises done.
            done = True
        self.lives = lives
        return obs, reward, done, info

    def reset(self, **kwargs):
        """Reset only when lives are exhausted.
        This way all states are still reachable even though lives are episodic,
        and the learner need not know about any of this behind-the-scenes.
        """
        if self.was_real_done:
            obs = self.env.reset(**kwargs)
        else:
            # no-op step to advance from terminal/lost life state
            obs, _, _, _ = self.env.step(0)
        self.lives = self.env.unwrapped.ale.lives()
        return obs


class MaxAndSkipEnv(gym.Wrapper):
    def __init__(self, env, skip=4):
        """Return only every `skip`-th frame"""
        gym.Wrapper.__init__(self, env)
        # most recent raw observations (for max pooling across time steps)
        shape = (2, ) + env.observation_space.shape
        self._obs_buffer = np.zeros(shape, dtype=np.uint8)
        self._skip = skip

    def step(self, action):
        """Repeat action, sum reward, and max over last observations."""
        total_reward = 0.0
        done = info = None
        for i in range(self._skip):
            obs, reward, done, info = self.env.step(action)
            if i == self._skip - 2:
                self._obs_buffer[0] = obs
            if i == self._skip - 1:
                self._obs_buffer[1] = obs
            total_reward += reward
            if done:
                break
        # Note that the observation on the done=True frame doesn't matter
        max_frame = self._obs_buffer.max(axis=0)

        return max_frame, total_reward, done, info

    def reset(self, **kwargs):
        return self.env.reset(**kwargs)


class ClipRewardEnv(gym.RewardWrapper):
    def __init__(self, env):
        gym.RewardWrapper.__init__(self, env)

    @staticmethod
    def reward(reward):
        """Bin reward to {+1, 0, -1} by its sign."""
        return np.sign(reward)


class WarpFrame(gym.ObservationWrapper):
    def __init__(self, env, width=84, height=84, grayscale=True):
        """Warp frames to 84x84 as done in the Nature paper and later work."""
        gym.ObservationWrapper.__init__(self, env)
        self.width = width
        self.height = height
        self.grayscale = grayscale
        shape = (1 if self.grayscale else 3, self.height, self.width)
        self.observation_space = spaces.Box(
            low=0, high=255, shape=shape, dtype=np.uint8
        )

    def observation(self, frame):
        if self.grayscale:
            frame = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
        size = (self.width, self.height)
        frame = cv2.resize(frame, size, interpolation=cv2.INTER_AREA)
        if self.grayscale:
            frame = np.expand_dims(frame, -1)
        return frame.transpose((2, 0, 1))


class FrameStack(gym.Wrapper):
    def __init__(self, env, k):
        """Stack k last frames.
        Returns lazy array, which is much more memory efficient.
        See Also `LazyFrames`
        """
        gym.Wrapper.__init__(self, env)
        self.k = k
        self.frames = deque([], maxlen=k)
        shp = env.observation_space.shape
        shape = (shp[0] * k, ) + shp[1:]
        self.observation_space = spaces.Box(
            low=0, high=255, shape=shape, dtype=env.observation_space.dtype
        )

    def reset(self):
        ob = self.env.reset()
        for _ in range(self.k):
            self.frames.append(ob)
        return np.asarray(self._get_ob())

    def step(self, action):
        ob, reward, done, info = self.env.step(action)
        self.frames.append(ob)
        return np.asarray(self._get_ob()), reward, done, info

    def _get_ob(self):
        assert len(self.frames) == self.k
        return LazyFrames(list(self.frames))


class LazyFrames(object):
    def __init__(self, frames):
        """This object ensures that common frames between the observations are
        only stored once. It exists purely to optimize memory usage which can be
        huge for DQN's 1M frames replay buffers.
        This object should only be converted to numpy array before being passed
        to the model. You'd not believe how complex the previous solution was.
        """
        self._frames = frames
        self._out = None

    def _force(self):
        if self._out is None:
            self._out = np.concatenate(self._frames, axis=-3)
            self._frames = None
        return self._out

    def __array__(self, dtype=None):
        out = self._force()
        if dtype is not None:
            out = out.astype(dtype)
        return out

    def __len__(self):
        return len(self._force())

    def __getitem__(self, i):
        return self._force()[i]

    
class ScaledFloatFrame(gym.ObservationWrapper):
    def __init__(self, env):
        gym.ObservationWrapper.__init__(self, env)
        self.observation_space = gym.spaces.Box(low=0, high=1, shape=env.observation_space.shape, dtype=np.float32)

    def observation(self, observation):
        # careful! This undoes the memory optimization, use
        # with smaller replay buffers only.
        return np.array(observation).astype(np.float32) / 255.0
    
    
def wrap_deepmind(env, episode_life=True, clip_rewards=True, frame_stack=False, scale=False):
    """Configure environment for DeepMind-style Atari.
    """
    if episode_life:
        env = EpisodicLifeEnv(env)
    if 'FIRE' in env.unwrapped.get_action_meanings():
        env = FireResetEnv(env)
    env = WarpFrame(env, width=84, height=84)
    env = MaxAndSkipEnv(env, skip=4)
    if scale:
        env = ScaledFloatFrame(env)
    if clip_rewards:
        env = ClipRewardEnv(env)
    if frame_stack:
        env = FrameStack(env, 4)
    return env

### Pong

In this notebook we'll train an agent to play the classic Atari game Pong. In Pong the agent controls a paddle and tries to hit a ball past the opponents paddle to score a point. The agent gets a positive reward when scoring a point and a negative reward when the opponent scores a point. The episode ends when a set number of points are reached.

In [143]:
#create Pong env and test it a bit
env = gym.make('PongNoFrameskip-v4', render_mode='rgb_array')
env.reset()

for i in range(3000):
    env.render()
    action = env.action_space.sample()
    next_state, reward, done, truncate, info = env.step(action)
    if done:
        env.reset()
env.close()

/home/schneialexan@edu.local/miniconda3/envs/rl/lib/python3.7/site-packages/gym/utils/passive_env_checker.py:290: UserWarning: WARN: No render fps was declared in the environment (env.metadata['render_fps'] is None or not defined), rendering may occur at inconsistent fps.
  "No render fps was declared in the environment (env.metadata['render_fps'] is None or not defined), rendering may occur at inconsistent fps."


### Deep Q Network

The Deep Q Network is from the original DQN paper and is sometimes referred to as the Nature CNN (since the article was published in Nature). Compared to state of the art CNNs in image recognition, the Nature CNN is quite simple. There are only three convolutional layers, followed by a fully connected (dense) layer and a linear layer. All but the linear layer have ReLU activations.

In [144]:
# create Deep Q Network Class by inheriting from torch.nn.Module
# based on Nature CNN from OpenAI baselines: https://github.com/openai/baselines/blob/1b092434fc51efcb25d6650e287f07634ada1e08/baselines/common/models.py
           
class DeepQNetwork(nn.Module):
    def __init__(self, action_size, hidden_size):
        super(DeepQNetwork, self).__init__()
        self.conv_layer_1 = nn.Conv2d(4, 32, kernel_size=8, stride=4)
        self.conv_layer_2 = nn.Conv2d(32, 64, kernel_size=4, stride=2)
        self.conv_layer_3 = nn.Conv2d(64, 64, kernel_size=3, stride=1)
        self.fc_layer = nn.Linear(7 * 7 * 64, hidden_size)
        # V(s) value of the state
        self.dueling_value = nn.Linear(hidden_size, 1)
        # Q(s,a) Q values of the state-action combination
        self.dueling_action = nn.Linear(hidden_size, action_size)
    
    def forward(self, x):
        x = F.relu(self.conv_layer_1(x))
        x = F.relu(self.conv_layer_2(x))
        x = F.relu(self.conv_layer_3(x))
        x = F.relu(self.fc_layer(x.view(x.size(0), -1)))
        # get advantage by subtracting dueling action mean from dueling action
            # then add estimated state value
        x = self.dueling_action(x) - self.dueling_action(x).mean(dim=1, keepdim=True) + self.dueling_value(x)
        return x

# Non-dueling network

# class DeepQNetwork(nn.Module):
#     def __init__(self, action_size, hidden_size):
#         super(DeepQNetwork, self).__init__()
#         self.conv_layer_1 = nn.Conv2d(4, 32, kernel_size=8, stride=4)
#         self.conv_layer_2 = nn.Conv2d(32, 64, kernel_size=4, stride=2)
#         self.conv_layer_3 = nn.Conv2d(64, 64, kernel_size=3, stride=1)
#         self.dense_layer = nn.Linear(7 * 7 * 64, hidden_size)
#         self.out_layer = nn.Linear(hidden_size, action_size)

#     def forward(self, x):
#         x = x / 255. # image data is stored as ints in 0 to 255 range. Divide to scale to 0 to 1 range
#         x = F.relu(self.conv_layer_1(x))
#         x = F.relu(self.conv_layer_2(x))
#         x = F.relu(self.conv_layer_3(x))
#         x = F.relu(self.dense_layer(x.view(x.size(0), -1)))
#         return self.out_layer(x)

### DQNAgent Class

We create a DQNAgent class. The agent has a train network for learning a policy and a target network for performing target network updates. The agent has a select_action() function for sampling an action based on the epsilon-greedy method. The agent has a train() function, in which the agent samples from the replay buffer and updates the neural network parameters to improve its policy. The agent also has a update_target_network() function for updating the target network parameters with the training network parameters.

In [145]:
class DQNAgent():
    def __init__(self, action_size, hidden_size, learning_rate ):
        self.action_size = action_size
        self.train_net = DeepQNetwork(action_size, hidden_size).to(device)
        self.target_net = DeepQNetwork(action_size, hidden_size).to(device)
        self.target_net.load_state_dict(self.train_net.state_dict())
        self.optimizer = optim.Adam(self.train_net.parameters(), lr=learning_rate)

        
    def select_action(self, s, eps, ts):
        # select action according to epsilon-greedy method
        if np.random.rand() <= eps:
            a = env.action_space.sample()
        else:
            print(ts, eps)
            # greedy action is the largest Q value from the train network based on the input
            with torch.no_grad():
                input_state = torch.FloatTensor(np.array(s)).unsqueeze(0).to(device)
                print(input_state.shape)
                a = self.train_net(input_state).max(1)[1]#.view(1, 1)#.detach().cpu().numpy()[0]
                a = int(a)
        return a

    
    def train(self, replay_buffer, batch_size, discount):
        # train the training network
        # sample a batch from the replay buffer
        x0, x1, a, r, d = replay_buffer.sample(batch_size)
        # turn batches into tensors and attack to GPU if available
        state_batch = torch.FloatTensor(x0).to(device)
        next_state_batch = torch.FloatTensor(x1).to(device)
        action_batch = torch.LongTensor(a).to(device)
        reward_batch = torch.FloatTensor(r).to(device)
        done_batch = torch.FloatTensor(1. - d).to(device)

        # get train net Q values
        train_q = self.train_net(state_batch).gather(1, action_batch)
        
        # get target net Q values
        with torch.no_grad():
            # Vanilla DQN: get target values from target net
#             target_net_q = reward_batch + done_batch * discount * \
#                      torch.max( self.target_net(next_state_batch).detach(), dim=1)[0].view(batch_size, -1)

            # Double DQN: get argmax values from train network, use argmax in target network
            train_argmax = self.train_net(next_state_batch).max(1)[1].view(batch_size, 1)
            target_net_q = reward_batch + done_batch * discount * \
                    self.target_net(next_state_batch).gather(1, train_argmax)
            
        # get loss between train q values and target q values
            # DQN implementations typically use MSE loss or Huber loss (smooth_l1_loss is similar to Huber)
        #loss_fn = nn.MSELoss()
        #loss = loss_fn(train_q, target_net_q) 
        loss = F.smooth_l1_loss(train_q, target_net_q)
        
        # optimize the parameters with the loss
        self.optimizer.zero_grad()
        loss.backward()
        for param in self.train_net.parameters():
            param.grad.data.clamp_(-1, 1)
        self.optimizer.step()
        # we return the loss so we can monitor loss and debug the network if necessary
        return loss.detach().cpu().numpy()   
    
    
    def update_target_network(self, num_iter, update_every):
        # update target network every so often
        # hard target network update: updates target network fully with train network params
        if num_iter % update_every == 0:
            #print("Updating target network parameters")
            self.target_net.load_state_dict(self.train_net.state_dict())    
        

    def update_target_network_soft(self, num_iter, update_every, update_tau=0.001):
        # soft target network update: update target network with mixture of train and target
        if num_iter % update_every == 0:
            for target_var, var in zip(self.target_net.parameters(), self.train_net.parameters()):
                target_var.data.copy_((1.-update_tau) * target_var.data + (update_tau) * var.data)

### Target Network Updating

DQN has two networks: a train network and a target network. The target network provides the next-state Q values for the Q learning update. We use the next-state Q values to update the train network. Periodically we then update target network variables with the train network variables. Using a target network increases stability and convergence of the DQN.

### Hyperparameters

Hyperparameters are a variety of arguments that are usually tuned to help produce the best performing agents. Hyperparameter choice can be tricky as poor hyperparameter choice can prevent an agent from learning anything. The best choice depends on the algorithm and problem. Two basic methods that are used are random search, where you randomly pick hyperparameter values from a predefined range of values, and grid search where you systematically try all values and combinations based on a predefined grid of values.

In [146]:
# Initialize env and set up hyperparameters
env = gym.make('PongNoFrameskip-v4')

# wrap env
#env = wrap_deepmind(env, episode_life=True, clip_rewards=True, frame_stack=True, scale=False)
action_size = env.action_space.n

# set seed
seed = 0
env.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

# hyperparameters
timesteps = 2000000 # run env for this many time steps
hidden_size = 512   # side of hidden layer of FFNN that connects CNN to outputs
learning_rate = 0.00001 # learning rate of optimizer
batch_size = 64    # size of batch trained on
start_training_after = 10001 # start training NN after this many timesteps
discount = 0.99 # discount future states by

epsilon_start = 1.0 # epsilon greedy start value
epsilon_min = 0.01  # epsilon greedy end value
epsilon_decay_steps = timesteps * .2 # decay epsilon over this many timesteps
epsilon_step = (epsilon_start - epsilon_min)/(epsilon_decay_steps) # decrement epsilon by this amount every timestep

update_target_every = 1 # update target network every this steps
tau = 0.001

# create replay buffer
replay_size = 100000 # size of replay buffer
replay_buffer = ReplayBuffer(max_size=replay_size)

# create DQN Agent
dqn_agent = DQNAgent(action_size, hidden_size, learning_rate)

### The RL Training Loop

In the loop, we iterate through our episodes and then the timesteps within the episodes. We stop training the agent based on some stopping condition like a max episode value, a max timestep value, or if the agent's performance reaches a certain level.

In each episode and at each timestep the agent selects an action, calls env.step(), and stores the sample in the replay buffer. The agent will also train the training neural network and periodically update the target network. When an episode is done the env is reset.

In [147]:
stats_rewards_list = [] # store stats for plotting in this
stats_every = 10 # print stats every this many episodes
total_reward = 0
episode = 1
episode_length = 0
stats_loss = 0.
epsilon = epsilon_start
state = env.reset()

for ts in range(timesteps):
    #env.render()
    # select an action from the agent's policy
    action = dqn_agent.select_action(state, epsilon, ts)
    # decay epsilon
    epsilon -= epsilon_step
    if epsilon < epsilon_min:
        epsilon = epsilon_min
            
    # enter action into the env
    next_state, reward, done, truncated, info = env.step(action)
    total_reward += reward
    episode_length += 1
    
    # add experience to replay buffer
    replay_buffer.add((state, next_state, action, reward, float(done)))
    
    if ts > start_training_after:
        # train the agent
        stats_loss += dqn_agent.train(replay_buffer, batch_size, discount)
        # update the target network every (if conditions are met in update_target_network)
        #dqn_agent.update_target_network(ts, update_target_every) # hard target update
        dqn_agent.update_target_network_soft(ts, update_target_every, tau) # soft target update
    
    if done:
        state = env.reset()
        stats_rewards_list.append((episode, total_reward, episode_length))
        episode += 1
        total_reward = 0
        episode_length = 0

        if ts > start_training_after and episode % stats_every == 0:
            print('Episode: {}'.format(episode),
                'Timestep: {}'.format(ts),
                'Total reward: {:.1f}'.format(np.mean(stats_rewards_list[-stats_every:],axis=0)[1]),
                'Episode length: {:.1f}'.format(np.mean(stats_rewards_list[-stats_every:],axis=0)[2]),
                'Epsilon: {:.2f}'.format(epsilon),
                'Loss: {:.4f}'.format(stats_loss))
            stats_loss = 0.
        
        # stopping condition for training if agent reaches the amount of reward
        if len(stats_rewards_list) > stats_every and np.mean(stats_rewards_list[-stats_every:],axis=0)[1] > 19:
            print("Stopping at episode {} with average rewards of {} in last {} episodes".
                format(episode, np.mean(stats_rewards_list[-stats_every:],axis=0)[1], stats_every))
            break  
    else:
        state = next_state
    

619 0.9984679750000333
torch.Size([1, 210, 160, 3])


RuntimeError: Given groups=1, weight of size [32, 4, 8, 8], expected input[1, 210, 160, 3] to have 4 channels, but got 210 channels instead

In [ ]:
# plot stats
def get_running_stat(stat, stat_len):
    cum_sum = np.cumsum(np.insert(stat, 0, 0)) 
    return (cum_sum[stat_len:] - cum_sum[:-stat_len]) / stat_len

episode, r, l = np.array(stats_rewards_list).T
cum_r = get_running_stat(r, 10)
cum_l = get_running_stat(l, 10)

# plot rewards
plt.plot(episode[-len(cum_r):], cum_r)
plt.plot(episode, r, alpha=0.5)
plt.xlabel('Episode')
plt.ylabel('Episode Reward')

In [ ]:
# plot episode lengths
plt.plot(episode[-len(cum_l):], cum_l)
plt.plot(episode, l, alpha=0.5)
plt.xlabel('Episode')
plt.ylabel('Episode Length')